In [1]:
"""
Function to compute the execution time.
"""
from datetime import timedelta
import time
def get_time(f, kwargs={}):
    times = []
    result = None
    for i in range(5):
        print("\nmeasure",i+1)
        t = time.time()
        result = f(**kwargs)
        res = time.time()-t
        times.append(res)
        print( timedelta(seconds=res))
        if res > 3600*2:
            print("trop de temps")
            return result
        
    print("\nall times:", times)
    a = min(times)
    b = max(times)
    print("times' outliers:",a,"&",b)
    t = (sum(times)-a-b)/3
    print("benchmark time",str(t)+"s","or", timedelta(seconds=t))
    return result

In [2]:
from tracktable.domain.terrestrial import TrajectoryPointReader
from tracktable.render.render_trajectories import render_trajectories, render_trajectories_separate
from tracktable.applications.assemble_trajectories import AssembleTrajectoryFromPoints
from tracktable.core.geomath import length, recompute_speed, simplify
import pandas as pd

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [11]:
SIZE = "large"

In [4]:
PATH = "datasets/ships_"+SIZE+".csv"

# Ships Trajectories

### Trajectories loading

In [ ]:
def load_ships_trajectories(PATH):
    ## Set up the point reader
    reader = TrajectoryPointReader()
    reader.input = open(PATH, 'r')
    reader.comment_character = '#'
    reader.field_delimiter = ','
    reader.null_value = 'NaN'
    reader.object_id_column = 3
    reader.timestamp_column = 2
    reader.coordinates[0] = 0     # longitude
    reader.coordinates[1] = 1     # latitude
    #reader.set_real_field_column('heading', 6)
    #reader.set_real_field_column('SOG', 4)
    #reader.set_string_field_column('vessel-name', 8)
    #reader.set_string_field_column('navigational status', 7)
    ## Put the point reader into the trajectoriesBuilder
    builder = AssembleTrajectoryFromPoints()
    builder.input = reader
    builder.separation_time = None
    print("building list")
    return list(builder)

trajectories = get_time(load_ships_trajectories,{'PATH':PATH})
print(len(trajectories))

INFO:tracktable.applications.assemble_trajectoriesAssembleTrajectoryFromPoints:New trajectories will be declared after a separation of None distance units between two points or a time lapse of at least None (hours, minutes, seconds).
INFO:tracktable.applications.assemble_trajectoriesAssembleTrajectoryFromPoints:Trajectories with fewer than 2 points will be discarded.



measure 1
building list


In [ ]:
traj = trajectories[1]
length(traj)

# Cars Trajectories

### Trajectories loading

In [12]:
PATH = "datasets/cars_"+SIZE+".csv"

In [13]:
def load_cars_trajectories(PATH):
    ## Set up the point reader
    reader = TrajectoryPointReader()
    reader.input = open(PATH, 'r')
    reader.comment_character = '#'
    reader.field_delimiter = ','
    reader.null_value = 'NaN'
    reader.object_id_column = 0
    reader.timestamp_column = 3
    reader.coordinates[0] = 1     # longitude
    reader.coordinates[1] = 2     # latitude
    
    ## Put the point reader into the trajectoriesBuilder
    builder = AssembleTrajectoryFromPoints()
    builder.input = reader
    builder.separation_time = None
    return list(builder)

trajectories = load_cars_trajectories(PATH)
print(len(trajectories))

INFO:tracktable.applications.assemble_trajectoriesAssembleTrajectoryFromPoints:New trajectories will be declared after a separation of None distance units between two points or a time lapse of at least None (hours, minutes, seconds).
INFO:tracktable.applications.assemble_trajectoriesAssembleTrajectoryFromPoints:Trajectories with fewer than 2 points will be discarded.


[2023-04-25 05:04:28.476436] [0x00007f60425e8440] [info]    Done reading points. Generated 10988601 points correctly and discarded 1 due to parse errors.



INFO:tracktable.applications.assemble_trajectoriesAssembleTrajectoryFromPoints:Done assembling trajectories. 447 trajectories produced and 0 discarded for having fewer than 2 points.


447


### Speed Computation

In [14]:
def speed_computation():
    for traj in trajectories:
        recompute_speed(traj)
get_time(speed_computation)

## There is not a good way to show the data, so we will put them into a pandas dataframe
import pandas
object_id    = []
timestamp    = []
longitude    = []
latitude     = []
speed        = []
SOG          = []
nav_stat     = []
df = pandas.DataFrame()
tt = time.time()
"""
for traj in trajectories:
    for trajectory_point in traj:
        object_id.append(trajectory_point.object_id)
        timestamp.append(trajectory_point.timestamp)
        longitude.append(trajectory_point[0])
        latitude.append(trajectory_point[1])
        speed.append(trajectory_point.properties["speed"])
        if PATH == "datasets/ships_sample.csv":
            SOG.append(trajectory_point.properties["SOG"])
            nav_stat.append(trajectory_point.properties["navigational status"])     
            df = pandas.DataFrame({"id":object_id,"timestamp":timestamp,"longitude":longitude,"latitude":latitude,"speed":speed,"SOG":SOG,"navigational status":nav_stat})
        else:
            df = pandas.DataFrame({"id":object_id,"timestamp":timestamp,"longitude":longitude,"latitude":latitude,"speed":speed})
df
print(time.time()-tt)"""


measure 1
0:00:26.181232

measure 2
0:00:26.008467

measure 3
0:00:26.224088

measure 4
0:00:26.210342

measure 5
0:00:26.096067

all times: [26.18123197555542, 26.008466720581055, 26.224088430404663, 26.210341691970825, 26.096067190170288]
times' outliers: 26.008466720581055 & 26.224088430404663
benchmark time 26.16254695256551s or 0:00:26.162547


'\nfor traj in trajectories:\n    for trajectory_point in traj:\n        object_id.append(trajectory_point.object_id)\n        timestamp.append(trajectory_point.timestamp)\n        longitude.append(trajectory_point[0])\n        latitude.append(trajectory_point[1])\n        speed.append(trajectory_point.properties["speed"])\n        if PATH == "datasets/ships_sample.csv":\n            SOG.append(trajectory_point.properties["SOG"])\n            nav_stat.append(trajectory_point.properties["navigational status"])     \n            df = pandas.DataFrame({"id":object_id,"timestamp":timestamp,"longitude":longitude,"latitude":latitude,"speed":speed,"SOG":SOG,"navigational status":nav_stat})\n        else:\n            df = pandas.DataFrame({"id":object_id,"timestamp":timestamp,"longitude":longitude,"latitude":latitude,"speed":speed})\ndf\nprint(time.time()-tt)'

In [15]:
df

""


### Compression

In [16]:
tolerance = 1.0 # meters

In [17]:
def compress_all(trajectories,tolerance):
    for trajectory in trajectories:
        compressed_traj = simplify(trajectory=trajectory, tolerance=tolerance / 111139)
get_time(compress_all,{"trajectories":trajectories,"tolerance":tolerance})


measure 1
0:00:48.048225

measure 2
0:00:48.024837

measure 3
0:00:48.033678

measure 4
0:00:47.997424

measure 5
0:00:48.004804

all times: [48.04822516441345, 48.024837017059326, 48.03367805480957, 47.99742364883423, 48.004804372787476]
times' outliers: 47.99742364883423 & 48.04822516441345
benchmark time 48.021106481552124s or 0:00:48.021106


### Visualisation

In [ ]:
get_time(render_trajectories,{"trajectories":trajectories})
#render_trajectories(trajectories)


measure 1


In [ ]:

#render_trajectories(compressed_traj)